In [ ]:
#ref:https://github.com/Elwing-Chou
'''條件機率；設定一個條件下，某件事情發生的機率
文字資料處理，將文章分詞後做成一向量

TF-IDF，
TF:這個詞出現在整篇文章的次數，出現越多次越能代表整篇文章
IDF:這個字在所有收集文章中出現過的文章數，出現越多次，代表這個詞是慣用詞，重要性下降
#https://drive.google.com/drive/folders/1zbVKIHMUugqXkKDc4q7CNaWXcwQ7pzFE

預測目標:用單貝氏預測一篇沒分類過的文章主題
1.分類問題
2.監督式演算法

'''

In [1]:
#資料預處理
#1.讀入訓練資料

import os
import jieba
import pandas as pd

#scikit-learn 會有 deprecation warning,為了顯示漂亮，先忽略

import warnings
warnings.filterwarnings('ignore')

base_dir= 'chinese_news_trans'
test_dir = 'chinese_news_test'


In [2]:
#os.walk遞迴印出資料夾中所有目錄及檔名

for dirPath, dirNames, fileNames in os.walk(base_dir):
    print('dirPath:',dirPath)
    #print('dirNames:',dirNames)
    #print('fileNames:',fileNames)
    for f in fileNames:
        print(os.path.join(dirPath, f))

dirPath: chinese_news_trans
chinese_news_trans\_DS_Store
dirPath: chinese_news_trans\交通
chinese_news_trans\交通\41.TXT
chinese_news_trans\交通\410.TXT
chinese_news_trans\交通\4100.TXT
chinese_news_trans\交通\4101.TXT
chinese_news_trans\交通\4102.TXT
chinese_news_trans\交通\4103.TXT
chinese_news_trans\交通\4104.TXT
chinese_news_trans\交通\4105.TXT
chinese_news_trans\交通\4106.TXT
chinese_news_trans\交通\4107.TXT
chinese_news_trans\交通\4108.TXT
chinese_news_trans\交通\4109.TXT
chinese_news_trans\交通\411.TXT
chinese_news_trans\交通\4110.TXT
chinese_news_trans\交通\4111.TXT
chinese_news_trans\交通\4112.TXT
chinese_news_trans\交通\4113.TXT
chinese_news_trans\交通\4114.TXT
chinese_news_trans\交通\4115.TXT
chinese_news_trans\交通\4116.TXT
chinese_news_trans\交通\4118.txt
chinese_news_trans\交通\4119.txt
chinese_news_trans\交通\412.TXT
chinese_news_trans\交通\4120.txt
chinese_news_trans\交通\4121.txt
chinese_news_trans\交通\4122.txt
chinese_news_trans\交通\4123.txt
chinese_news_trans\交通\4124.txt
chinese_news_trans\交通\4125.txt
chinese_news_trans

chinese_news_trans\體育\10324.txt
chinese_news_trans\體育\10325.txt
chinese_news_trans\體育\10326.txt
chinese_news_trans\體育\10327.txt
chinese_news_trans\體育\10328.txt
chinese_news_trans\體育\10329.txt
chinese_news_trans\體育\1033.txt
chinese_news_trans\體育\10330.txt
chinese_news_trans\體育\10331.txt
chinese_news_trans\體育\10332.txt
chinese_news_trans\體育\10333.txt
chinese_news_trans\體育\10334.txt
chinese_news_trans\體育\10335.txt
chinese_news_trans\體育\10336.txt
chinese_news_trans\體育\10337.txt
chinese_news_trans\體育\10338.txt
chinese_news_trans\體育\10339.txt
chinese_news_trans\體育\1034.txt
chinese_news_trans\體育\10340.txt
chinese_news_trans\體育\10341.txt
chinese_news_trans\體育\10342.txt
chinese_news_trans\體育\10343.txt
chinese_news_trans\體育\10344.txt
chinese_news_trans\體育\10345.txt
chinese_news_trans\體育\10346.txt
chinese_news_trans\體育\10347.txt
chinese_news_trans\體育\10348.txt
chinese_news_trans\體育\10349.txt
chinese_news_trans\體育\1035.txt
chinese_news_trans\體育\10350.txt
chinese_news_trans\體育\10351.txt
chinese_new

In [3]:
#輸入資料集
def process_dirs(base_dir):
    df = pd.DataFrame(columns=["類別", "內容"])
    
    for dir_path, dir_name, file_names in os.walk(base_dir):
        for single_file in file_names:
                try:
                    f = open(os.path.join(dir_path, single_file), "r", encoding="utf-8")
                    content = f.read()
                    content = content.replace("\r",'').replace("\n", '')
                    split_word = jieba.cut(content)
                    content = ' '.join(split_word)
                    s = pd.Series([dir_path.split("\\")[1],content], index=["類別","內容"])
                    df = df.append(s, ignore_index = True)
                except UnicodeDecodeError:
                    pass
    df['類別'] = df['類別'].astype('category')
    return df



In [4]:
df = process_dirs(base_dir)
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\keneau\AppData\Local\Temp\jieba.cache
Loading model cost 0.830 seconds.
Prefix dict has been built succesfully.


,類別,內容
0,交通,【 日 期 】 19960104 【 版 號 】 1 【 ...
1,交通,【 日 期 】 19960226 【 版 號 】 5 【 ...
2,交通,大 秦鐵路 萬噸 列車 試運 成功 新華社 北京...
3,交通,遼寧省 檯 安縣 村村 都 通 柏油路 鄉村 公...
4,交通,北京 — 烏蘭 巴托 — 莫斯料 ３ ／ ４ 次...


In [5]:
#類別處理,scikit-learn不接受字串，須把類別轉換成整數

saved_map = {cat:df['類別'].cat.categories.get_loc(cat) for cat in df['類別'].cat.categories}
saved_map

{'交通': 0,
 '政治': 1,
 '教育': 2,
 '環境': 3,
 '經濟': 4,
 '藝術': 5,
 '計算機': 6,
 '軍事': 7,
 '醫藥': 8,
 '體育': 9}

In [6]:
#轉換整數:#https://zhuanlan.zhihu.com/p/44256257
df['類別'] = df['類別'].cat.codes
df.head()

,類別,內容
0,0,【 日 期 】 19960104 【 版 號 】 1 【 ...
1,0,【 日 期 】 19960226 【 版 號 】 5 【 ...
2,0,大 秦鐵路 萬噸 列車 試運 成功 新華社 北京...
3,0,遼寧省 檯 安縣 村村 都 通 柏油路 鄉村 公...
4,0,北京 — 烏蘭 巴托 — 莫斯料 ３ ／ ４ 次...


In [7]:
#讀入資料
test_df = process_dirs(test_dir)
test_df.head()

,類別,內容
0,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
1,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
2,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...
3,交通,三趟 火車 停開 乘客 可 全額 退票 瀏覽次數 ： 1180 ...
4,交通,日 月光 華 - - Traffic _ Info 精華區 文章 閱讀 - - - ...


In [8]:
#類別轉成數值
test_df['類別']= test_df['類別'].replace(saved_map)

In [9]:

from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
#使用 fit_transform，才會轉成詞向量
bag = vec.fit_transform(df['內容'])
print("總共維度:",len(vec.get_feature_names()))
print(bag)
#transform後的matrix
#(0, 18254) 0.0915545916432285:(文章, 詞) tf-idf權重


總共維度: 96042
  (0, 18254)	0.0915545916432285
  (0, 55034)	0.17977354410092267
  (0, 80225)	0.17086759391872436
  (0, 10614)	0.07545688183872995
  (0, 18245)	0.09930423402371821
  (0, 53311)	0.17086759391872436
  (0, 17465)	0.05696876874041001
  (0, 39510)	0.09236072893655461
  (0, 66841)	0.08379960178230576
  (0, 11866)	0.12103646864033255
  (0, 83223)	0.1281261138259475
  (0, 38839)	0.12429648077255365
  (0, 88593)	0.057299072344780516
  (0, 86457)	0.1645487217042411
  (0, 71141)	0.08508780792185333
  (0, 11941)	0.17977354410092267
  (0, 78886)	0.0903982838415199
  (0, 27901)	0.05104268886595841
  (0, 60873)	0.14442260123839737
  (0, 44126)	0.08436283202092827
  (0, 85725)	0.17977354410092267
  (0, 48865)	0.09958765441760986
  (0, 33570)	0.11887425451419638
  (0, 39474)	0.28465826902077346
  (0, 68368)	0.25038625345735926
  :	:
  (2636, 22951)	0.09343610233808236
  (2636, 49200)	0.08998073214556616
  (2636, 64559)	0.14986741093052494
  (2636, 36963)	0.32096236251787386
  (2636, 48186)	

In [10]:
bag
#2637文章

<2637x96042 sparse matrix of type '<class 'numpy.float64'>'
	with 458826 stored elements in Compressed Sparse Row format>

In [ ]:
'''
Naive Bayes:單純貝氏，機率演算法，利用機率來推斷在某個特徵下，是某種分類的機率
條件機率:𝑃("屬於某類|"具某特徵")=(𝑃("具某特徵" │"屬於某類" )𝑃("屬於某類"))/(𝑃("具某特徵"))


'''

In [11]:

#scikit-learn單純貝氏,BernoulliNB:對於特徵True 或False二分優化,GaussianNB:對於特徵是高斯分布的連續數字優化,MultinomialNB:對於特徵是整數，而且需數幾次的分布優化     
#只要是文字，通常選擇MultinomialNB
#alpha:在矩陣中許多元素為0，造成條件機率:𝑃("具某特徵" │"屬於某類" )=0，造成預測過於武斷，降低準確率
#alpha:alpha不設定預設為1，但觀察我們tf-idf的數值大約多落在0.?中，所以一般設置為aplha=0.001
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.001).fit(bag, df['類別'])

In [12]:
#使用剛剛的vec所以維度相同
test_bag = vec.transform(test_df['內容'])
#print(test_bag)
print("維度:",len(vec.get_feature_names()))

維度: 96042


In [13]:
from sklearn.metrics import accuracy_score
predict = clf.predict(test_bag)
print("預測:",list(predict))
print("正確標籤:",list(test_df['類別']))
print('Naive-Bayes 正確率',accuracy_score(test_df['類別'],predict)*100,"%")

預測: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
Naive-Bayes 正確率 100.0 %


In [14]:
#試試KNN
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors = 8)
clf = clf.fit(bag, df['類別'])
predict = clf.predict(test_bag)
print("預測:",list(predict))
print("正確標籤:",list(test_df['類別']))
print('KNN 正確率',accuracy_score(test_df['類別'],predict)*100,"%")

預測: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
KNN 正確率 100.0 %
